# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import sys
import os
from pprint import pprint

# Import API key
sys.path.append("..")
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read in the csv file containing the data generated in the WeatherPy script
# create the path
csv_data_file = os.path.join("..", "WeatherPy", "output_data", "cities.csv")

# open the csv file to read
with open(csv_data_file, 'r', encoding='utf-8') as read_file:
    
    #extract the data from the csv and store into a DataFrame
    data_df = pd.read_csv(read_file, na_filter = False)
    
# close the csv file
read_file.close()

# display the data in the DataFrame
data_df

,City,Latitude,Longitude,Max Temp(F),Humidity(%),Cloudiness(%),Wind Speed (mph),Country,Date
0,Rikitea,-23.12,-134.97,69.04,59,8,19.82,PF,1593537784
1,Provideniya,64.38,-173.30,43.30,87,100,8.39,RU,1593537803
2,Havre-St-Pierre,50.23,-63.60,71.60,64,1,10.29,CA,1593537803
3,Cape Town,-33.93,18.42,60.01,87,12,3.36,ZA,1593537523
4,Chumikan,54.72,135.31,45.01,96,100,10.25,RU,1593537803
...,...,...,...,...,...,...,...,...,...
554,Karasburg,-28.02,18.75,56.39,33,0,8.32,NA,1593537850
555,Viedma,-40.81,-63.00,49.26,68,100,6.91,AR,1593537850
556,Shenjiamen,29.96,122.30,72.05,87,62,3.89,CN,1593537850
557,Bilibino,68.05,166.44,40.84,81,100,3.74,RU,1593537850


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Pass the api key to the Google Maps API
gmaps.configure(api_key=g_key)

# store the coordinates from the DataFrame into a list (for passing longitude and latitude to gmaps)
locations = data_df[['Latitude', 'Longitude']]

# story the humidity values from the DataFrame into a list (for passing to gmaps for the heatmap value)
humidity = data_df['Humidity(%)'].astype(float)

In [4]:
# Create the gmaps figure & settings
fig = gmaps.figure(zoom_level=1, center=[0,0], map_type='TERRAIN', layout={'width': '600px', 'height': '500px'})

# Generate the heatmap layer based on the humidity
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100, point_radius=3)

# Add the heatmap layer to the figure
fig.add_layer(heat_layer)

# display the gmaps figure
fig

Figure(layout=FigureLayout(height='500px', width='600px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# I prefer winds between 5 - 15 mph, I prefer cooler temperatures between 60 - 75, Low Cloudiness
prefer_df = data_df.loc[(data_df['Wind Speed (mph)'] >= 7) & (data_df['Wind Speed (mph)'] < 15) & 
                       (data_df['Max Temp(F)'] >= 60) & (data_df['Max Temp(F)'] < 75) & 
                       (data_df['Cloudiness(%)'] < 10)]

# Display the number of records that meet the criteria given above
print(len(prefer_df))

# Display the DataFrame containing the records that meet the criteria above
prefer_df

15


,City,Latitude,Longitude,Max Temp(F),Humidity(%),Cloudiness(%),Wind Speed (mph),Country,Date
2,Havre-St-Pierre,50.23,-63.60,71.60,64,1,10.29,CA,1593537803
32,Guerrero Negro,27.98,-114.06,72.25,57,2,9.78,MX,1593537557
48,Port Elizabeth,-33.92,25.57,60.80,77,0,8.05,ZA,1593537807
53,East London,-33.02,27.91,61.79,64,1,7.87,ZA,1593537808
93,Lompoc,34.64,-120.46,64.99,77,0,8.05,US,1593537707
116,Oranjemund,-28.55,16.43,69.66,29,0,8.46,NA,1593537812
135,Mossamedes,-15.20,12.15,70.65,82,0,8.21,AO,1593537814
189,Zhigansk,66.77,123.37,66.92,54,3,8.39,RU,1593537784
336,Ankazoabo,-22.28,44.52,66.60,56,0,12.21,MG,1593537830
385,Isangel,-19.55,169.27,67.69,67,0,13.91,VU,1593537835


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Copy the DataFrame created above to another DataFrame that will be changed
hotel_df = prefer_df

# Add a column to the new DataFrame to hold the names of the Hotels that will be pulled with the API
hotel_df['Hotel Name'] = ''

# Define the url required for the API
base_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

# Define the required parameters to send with the API request
params = {'radius': 5000,
         'type': 'hotel',
         'key': g_key}

# Define a list to temporarily hold the hotel names as we loop through the cities in the DataFrame
hotel_names = []

# Loop through the rows of the DataFrame
for index, row in hotel_df.iterrows():
    # Defind the location for the current row using the Latitude & Longitude columns in the DataFrame
    loc = f'{row["Latitude"]},{row["Longitude"]}'
    
    # Define a new parameter key:value pair for the current row
    params['location'] = loc
    
    # Display the city name to the user that is currently being processed
    print(f'Retrieving Results for Index {index}: {row["City"]}.')

    # Make the API request with a json format return
    response_json = requests.get(base_url, params=params).json()
    
    # store the API call results into another variable
    results = response_json['results']

    # loop through the results to find the first hotel that had an "operational" status, so no closed hotels are returned
    for i in range(0, len(results)):
        try:

            if results[i]['business_status'] == 'OPERATIONAL':
                print (f'Closest hotel is {results[i]["name"]}')

                # Add the hotel name to the list
                hotel_names.append(results[i]["name"])
                break
        except:
            i += 1
    print('--------------------------------')
    
# Copy the list of hotel names generated with the loop above into the column of the DataFrame
hotel_df['Hotel Name'] = hotel_names

Retrieving Results for Index 2: Havre-St-Pierre.


C:\Users\askbe\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Closest hotel is Auberge Niapiskau
--------------------------------
Retrieving Results for Index 32: Guerrero Negro.
Closest hotel is Bimbo
--------------------------------
Retrieving Results for Index 48: Port Elizabeth.
Closest hotel is Gelvandale Stadium
--------------------------------
Retrieving Results for Index 53: East London.
Closest hotel is Tu Casa
--------------------------------
Retrieving Results for Index 93: Lompoc.
Closest hotel is Holiday Inn Express Lompoc
--------------------------------
Retrieving Results for Index 116: Oranjemund.
Closest hotel is PEP Oranjemund
--------------------------------
Retrieving Results for Index 135: Mossamedes.
Closest hotel is Pavilhão Welwitschia Mirabilis
--------------------------------
Retrieving Results for Index 189: Zhigansk.
Closest hotel is Kapital Meditsinskoye Strakhovaniye
--------------------------------
Retrieving Results for Index 336: Ankazoabo.
Closest hotel is FJKM Ankazoabo
--------------------------------
Retrievin

C:\Users\askbe\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
# Display the DataFrame, showing the added hotel names
hotel_df

,City,Latitude,Longitude,Max Temp(F),Humidity(%),Cloudiness(%),Wind Speed (mph),Country,Date,Hotel Name
2,Havre-St-Pierre,50.23,-63.60,71.60,64,1,10.29,CA,1593537803,Auberge Niapiskau
32,Guerrero Negro,27.98,-114.06,72.25,57,2,9.78,MX,1593537557,Bimbo
48,Port Elizabeth,-33.92,25.57,60.80,77,0,8.05,ZA,1593537807,Gelvandale Stadium
53,East London,-33.02,27.91,61.79,64,1,7.87,ZA,1593537808,Tu Casa
93,Lompoc,34.64,-120.46,64.99,77,0,8.05,US,1593537707,Holiday Inn Express Lompoc
116,Oranjemund,-28.55,16.43,69.66,29,0,8.46,NA,1593537812,PEP Oranjemund
135,Mossamedes,-15.20,12.15,70.65,82,0,8.21,AO,1593537814,Pavilhão Welwitschia Mirabilis
189,Zhigansk,66.77,123.37,66.92,54,3,8.39,RU,1593537784,Kapital Meditsinskoye Strakhovaniye
336,Ankazoabo,-22.28,44.52,66.60,56,0,12.21,MG,1593537830,FJKM Ankazoabo
385,Isangel,-19.55,169.27,67.69,67,0,13.91,VU,1593537835,Tanna Lodge


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name 
# (my dataframe was already named correctly, but I did have to update the columns names to match mine)
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map, include popup boxes with more information about each marker
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='500px', width='600px'))